<a href="https://colab.research.google.com/github/ritamgh/DLT-lab/blob/main/Week12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# =======================
# Lab 12: DCGAN on CIFAR-10
# =======================
!pip -q install torch torchvision torchaudio torchinfo

import os, math, time
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, utils
from torch.utils.data import DataLoader

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Device:", device)

# --- Config
batch_size = 128
image_size = 32
nz = 100       # latent dim
ngf = 64       # generator feature maps
ndf = 64       # discriminator feature maps
nc = 3         # RGB
epochs = 10    # bump to 50+ for better results
out_dir = "/content/dcgan_samples"
os.makedirs(out_dir, exist_ok=True)

# --- Data
transform = transforms.Compose([
    transforms.Resize(image_size),
    transforms.CenterCrop(image_size),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3),  # [-1,1]
])
trainset = datasets.CIFAR10(root="/content/data", train=True, download=True, transform=transform)
loader = DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)

# --- Generator
class Generator(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            # input Z: (nz) -> (ngf*4) x 4 x 4
            nn.ConvTranspose2d(nz, ngf*4, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf*4),
            nn.ReLU(True),
            # -> (ngf*2) x 8 x 8
            nn.ConvTranspose2d(ngf*4, ngf*2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf*2),
            nn.ReLU(True),
            # -> (ngf) x 16 x 16
            nn.ConvTranspose2d(ngf*2, ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            # -> (nc) x 32 x 32
            nn.ConvTranspose2d(ngf, nc, 4, 2, 1, bias=False),
            nn.Tanh()
        )
    def forward(self, z): return self.net(z)

# --- Discriminator
class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            # (nc) x 32 x 32 -> (ndf) x 16 x 16
            nn.Conv2d(nc, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            # -> (ndf*2) x 8 x 8
            nn.Conv2d(ndf, ndf*2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf*2),
            nn.LeakyReLU(0.2, inplace=True),
            # -> (ndf*4) x 4 x 4
            nn.Conv2d(ndf*2, ndf*4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf*4),
            nn.LeakyReLU(0.2, inplace=True),
            # -> 1 x 1 x 1
            nn.Conv2d(ndf*4, 1, 4, 1, 0, bias=False)
        )
    def forward(self, x): return self.net(x).view(-1)

G = Generator().to(device)
D = Discriminator().to(device)
print(f"Generator params: {sum(p.numel() for p in G.parameters())/1e6:.2f}M")
print(f"Discriminator params: {sum(p.numel() for p in D.parameters())/1e6:.2f}M")

# --- Loss/Opt
criterion = nn.BCEWithLogitsLoss()
optG = optim.Adam(G.parameters(), lr=2e-4, betas=(0.5, 0.999))
optD = optim.Adam(D.parameters(), lr=2e-4, betas=(0.5, 0.999))

# --- Training
fixed_z = torch.randn(64, nz, 1, 1, device=device)

step = 0
for epoch in range(1, epochs+1):
    for real, _ in loader:
        real = real.to(device)
        b = real.size(0)
        # Real/Fake labels (with a tiny smoothing)
        real_lbl = torch.ones(b, device=device) * 0.9
        fake_lbl = torch.zeros(b, device=device)

        # ---- Train D
        D.zero_grad()
        out_real = D(real)
        loss_real = criterion(out_real, real_lbl)

        z = torch.randn(b, nz, 1, 1, device=device)
        fake = G(z).detach()
        out_fake = D(fake)
        loss_fake = criterion(out_fake, fake_lbl)

        lossD = loss_real + loss_fake
        lossD.backward()
        optD.step()

        # ---- Train G
        G.zero_grad()
        z = torch.randn(b, nz, 1, 1, device=device)
        gen = G(z)
        out = D(gen)
        lossG = criterion(out, real_lbl)   # want D to call fakes "real"
        lossG.backward()
        optG.step()

        step += 1

    # Save sample grid each epoch
    with torch.no_grad():
        samples = G(fixed_z).cpu()
        samples = (samples+1)/2  # back to [0,1]
        utils.save_image(samples, f"{out_dir}/epoch_{epoch:03d}.png", nrow=8)
    print(f"Epoch {epoch}/{epochs} | lossD={lossD.item():.3f} | lossG={lossG.item():.3f} | saved {out_dir}/epoch_{epoch:03d}.png")

print("Done. Sample images saved in:", out_dir)

Device: cpu


100%|██████████| 170M/170M [00:04<00:00, 41.3MB/s]


Generator params: 1.07M
Discriminator params: 0.66M


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch 1/10 | lossD=0.529 | lossG=2.940 | saved /content/dcgan_samples/epoch_001.png
Epoch 2/10 | lossD=0.723 | lossG=3.075 | saved /content/dcgan_samples/epoch_002.png
Epoch 3/10 | lossD=0.845 | lossG=2.977 | saved /content/dcgan_samples/epoch_003.png
Epoch 4/10 | lossD=0.568 | lossG=2.535 | saved /content/dcgan_samples/epoch_004.png
Epoch 5/10 | lossD=0.532 | lossG=1.974 | saved /content/dcgan_samples/epoch_005.png
Epoch 6/10 | lossD=1.113 | lossG=1.498 | saved /content/dcgan_samples/epoch_006.png
Epoch 7/10 | lossD=0.574 | lossG=2.339 | saved /content/dcgan_samples/epoch_007.png
Epoch 8/10 | lossD=0.789 | lossG=2.108 | saved /content/dcgan_samples/epoch_008.png
Epoch 9/10 | lossD=0.797 | lossG=2.860 | saved /content/dcgan_samples/epoch_009.png
Epoch 10/10 | lossD=0.938 | lossG=0.809 | saved /content/dcgan_samples/epoch_010.png
Done. Sample images saved in: /content/dcgan_samples
